In [1]:
import numpy as np
import pandas as pd
import nltk
from nltk.corpus import stopwords
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import cohen_kappa_score
from textblob import TextBlob
import language_check
from sklearn.model_selection import StratifiedKFold
from sklearn import svm
from sklearn.ensemble import RandomForestRegressor
from keras.layers import Embedding, LSTM, Dense, Dropout, Lambda, Flatten
from keras.models import Sequential, load_model, model_from_config
import keras.backend as K
import re
from gensim.models import Word2Vec
from sklearn.model_selection import KFold
from sklearn.linear_model import LinearRegression
from sklearn.metrics import cohen_kappa_score

stopwords = set(stopwords.words('english'))

Using TensorFlow backend.
/home/ayushi/anaconda3/lib/python3.6/site-packages/smart_open/ssh.py:34: UserWarning: paramiko missing, opening SSH/SCP/SFTP paths will be disabled.  `pip install paramiko` to suppress
  warnings.warn('paramiko missing, opening SSH/SCP/SFTP paths will be disabled.  `pip install paramiko` to suppress')


In [2]:
import enchant
from nltk.tokenize import RegexpTokenizer

In [3]:
df = pd.read_excel('test2.xlsx')
df.head(2)

,essay_id,essay_set,essay,domain1_score
0,1,1,"Dear local newspaper, I think effects computer...",6.666667
1,2,1,"Dear @CAPS1 @CAPS2, I believe that using compu...",7.500000


In [4]:
import json

num_rows = df.shape[0]
essays = df['essay'].values

#Initialize dataframe columns
df['word_count'] = np.nan 
df['sentence_count'] = np.nan
df['avg_word_length'] = np.nan 
df['num_exclamation_marks'] = np.nan
df['num_question_marks'] = np.nan
df['num_stopwords'] = np.nan

df['noun_count'] = np.nan
df['verb_count'] = np.nan
df['foreign_count'] = np.nan
df['adj_count'] = np.nan
df['conj_count'] = np.nan
df['adv_count'] = np.nan
df['beauty_score'] = np.nan
df['maturity_score'] = np.nan
df['vocab'] = np.nan
def get_pos_tags(essay):
    nouns = verbs = foreign = adj = adv = conj = 0
    tokens = nltk.word_tokenize(essay)
    for token in tokens:
        pos_tag = nltk.pos_tag(nltk.word_tokenize(token))
        for (_, tag) in (pos_tag):
            if tag[0] == "N":
                nouns += 1
            elif tag[0] == "V":
                verbs += 1
            elif tag[0:2] == "FW":
                foreign += 1
            elif tag[0] == "J":
                adj += 1
            elif tag[0] == "R":
                adv += 1
            elif tag[0:2] == "CC" or tag[0:2] == "IN":
                conj += 1
    
    return [nouns, verbs, foreign, adj, adv, conj]


for i in range(num_rows):
    
    # Turn essay into list of words
    text = essays[i].split(" ")
    
    # Set word count
    df.set_value(i,'word_count', len(text))
    
    # Sentence count
    df.set_value(i, 'sentence_count', len(nltk.tokenize.sent_tokenize(essays[i])))
    
    # Average word length
    word_len = sum(len(word) for word in text) / len(text)
    df.set_value(i, 'avg_word_length', word_len)
    
    # Number of exclamation marks
    df.set_value(i, "num_exclamation_marks", sum(word.count("!") for word in essays[i]))
    
    # Number of question marks
    df.set_value(i, "num_question_marks", sum(word.count("?") for word in essays[i]))
    
    # Number of stop words
    df.set_value(i, "num_stopwords", sum([1 for word in text if word.lower() in stopwords]))

    
    # POS tag counts
    pos_lst = get_pos_tags(essays[i])
    df.set_value(i,'noun_count', pos_lst[0])
    df.set_value(i,'verb_count', pos_lst[1])
    df.set_value(i,'foreign_count', pos_lst[2])
    df.set_value(i,'adj_count', pos_lst[3])
    df.set_value(i,'adv_count', pos_lst[4])
    df.set_value(i,'conj_count', pos_lst[5])
def avg_sentence_sentiment(x):
    sentiment_essay = TextBlob(x).sentiment.polarity
    return sentiment_essay
df['sentiment_essay'] = df['essay'].apply(avg_sentence_sentiment)
def grammar_check(x):
    tool = language_check.LanguageTool('en-US')
    matches = tool.check(x)
    return len(matches)
df['Grammar_check'] = df['essay'].apply(grammar_check)


print (df.head())


   essay_id  essay_set                                              essay  \
0         1          1  Dear local newspaper, I think effects computer...   
1         2          1  Dear @CAPS1 @CAPS2, I believe that using compu...   
2         3          1  Dear, @CAPS1 @CAPS2 @CAPS3 More and more peopl...   
3         4          1  Dear Local Newspaper, @CAPS1 I have found that...   
4         5          1  Dear @LOCATION1, I know having computers has a...   

   domain1_score  word_count  sentence_count  avg_word_length  \
0       6.666667       338.0            16.0         4.550296   
1       7.500000       419.0            20.0         4.463007   
2       5.833333       279.0            14.0         4.526882   
3       8.333333       524.0            27.0         5.041985   
4       6.666667       465.0            30.0         4.526882   

   num_exclamation_marks  num_question_marks  num_stopwords      ...        \
0                    4.0                 2.0          168.0      ...

In [5]:
temp = json.load(open('aoa_values.json'))
dataform = str(temp).strip("'<>()").replace('\'','\"')
words_acq_age = json.loads(dataform)

# words_acq_age = json.loads('aoa_values.json')
enchant_dict = enchant.Dict("en_US")
beauty_reference = {
    'a' : 8.12, 'b' : 1.49, 'c' : 2.71, 'd' : 4.32, 'e' : 12.02, 'f' : 2.30,
    'g' : 2.03, 'h' : 5.92, 'i' : 7.31, 'j' : 0.10, 'k' : 0.69, 'l' : 3.98,
    'm' : 2.61, 'n' : 6.95, 'o' : 7.68, 'p' : 1.82, 'q' : 0.11, 'r' : 6.02,
    's' : 1.68, 't' : 9.10, 'u' : 2.88, 'v' : 1.11, 'w' : 2.09, 'x' : 0.17,
    'y' : 2.11, 'z' : 0.07,
}


def find_BScore(essay):
    bs=0
    tokenizer= RegexpTokenizer(r'\w+')
    words=tokenizer.tokenize(essay)
    for word in words:
        s = 1.0
        for letter in word:
           try:
             s = s*beauty_reference[letter.lower()]
           except:
             pass
           bs += 1/s
    return bs

BS=[]
for i in range(num_rows):
    BS.append(find_BScore(essays[i]))
df.beauty_score=BS

        
def find_MScore(essay):        
   vocab = 0
   ms=0
   tokenizer= RegexpTokenizer(r'\w+')
   words=tokenizer.tokenize(essay)
   for word in words:
       lower_word = word.lower()
       if lower_word in words_acq_age and len(lower_word) > 3:
             if words_acq_age[lower_word] != "NA":
                 ms = ms + float(words_acq_age[lower_word])
                 vocab += 1
       if ms>0:
         ms /= vocab
   return ms
        
MS=[]
for i in range(num_rows):
    MS.append(find_MScore(essays[i]))
df.maturity_score=MS

def voc(essay):        
   vocab = 0
   tokenizer= RegexpTokenizer(r'\w+')
   words=tokenizer.tokenize(essay)
   for word in words:
       lower_word = word.lower()
       if lower_word in words_acq_age and len(lower_word) > 3:
             vocab += 1
   return vocab
        
V=[]
for i in range(num_rows):
    V.append(voc(essays[i]))
df.vocab=V

In [6]:
df.head()

,essay_id,essay_set,essay,domain1_score,word_count,sentence_count,avg_word_length,num_exclamation_marks,num_question_marks,num_stopwords,...,verb_count,foreign_count,adj_count,conj_count,adv_count,beauty_score,maturity_score,vocab,sentiment_essay,Grammar_check
0,1,1,"Dear local newspaper, I think effects computer...",6.666667,338.0,16.0,4.550296,4.0,2.0,168.0,...,39.0,0.0,13.0,69.0,22.0,175.314893,0.029508,183,0.310471,11
1,2,1,"Dear @CAPS1 @CAPS2, I believe that using compu...",7.500000,419.0,20.0,4.463007,1.0,1.0,189.0,...,56.0,0.0,14.0,80.0,21.0,239.987278,0.023986,217,0.274000,19
2,3,1,"Dear, @CAPS1 @CAPS2 @CAPS3 More and more peopl...",5.833333,279.0,14.0,4.526882,0.0,0.0,140.0,...,33.0,0.0,13.0,50.0,17.0,132.356485,0.060025,146,0.340393,9
3,4,1,"Dear Local Newspaper, @CAPS1 I have found that...",8.333333,524.0,27.0,5.041985,2.0,1.0,222.0,...,57.0,0.0,29.0,84.0,30.0,239.253012,0.019373,271,0.266828,35
4,5,1,"Dear @LOCATION1, I know having computers has a...",6.666667,465.0,30.0,4.526882,0.0,0.0,236.0,...,60.0,0.0,18.0,63.0,41.0,260.246296,0.021028,249,0.199684,17


In [7]:
from pandas import ExcelWriter
writer = ExcelWriter('features.xlsx')
df.to_excel(writer,'Sheet1',index=False)
writer.save()

In [15]:
from sklearn import preprocessing

min_max_scaler = preprocessing.MinMaxScaler()

df = df.fillna(0);
# where_are_NaNs = np.isnan(df)
# df[where_are_NaNs] = 0

np_scaled = min_max_scaler.fit_transform(df.drop(['essay','essay_id','essay_set'],axis=1))
df_normalized = pd.DataFrame(np_scaled, columns = ['domain1_score','word_count','sentence_count','avg_word_len','num_exclamation_marks','num_question_marks','num_stopwords','noun_count','verb_count','foreign_count','adj_count','conj_count','adv_count','beauty_score','maturity_score','vocab','sentiment_essay','Grammar_check'])
df_normalized.head(5)

,domain1_score,word_count,sentence_count,avg_word_len,num_exclamation_marks,num_question_marks,num_stopwords,noun_count,verb_count,foreign_count,adj_count,conj_count,adv_count,beauty_score,maturity_score,vocab,sentiment_essay,Grammar_check
0,0.666667,0.300806,0.157895,0.222340,0.148148,0.014493,0.296625,0.255864,0.209677,0.0,0.188406,0.328571,0.207547,0.053007,0.003472,0.300330,0.655235,0.085938
1,0.750000,0.373321,0.200000,0.216791,0.037037,0.007246,0.333925,0.315565,0.301075,0.0,0.202899,0.380952,0.198113,0.072601,0.002822,0.356436,0.637000,0.148438
2,0.583333,0.247986,0.136842,0.220851,0.000000,0.000000,0.246892,0.234542,0.177419,0.0,0.188406,0.238095,0.160377,0.039991,0.007062,0.239274,0.670196,0.070312
3,0.833333,0.467323,0.273684,0.253594,0.074074,0.007246,0.392540,0.560768,0.306452,0.0,0.420290,0.400000,0.283019,0.072378,0.002279,0.445545,0.633414,0.273438
4,0.666667,0.414503,0.305263,0.220851,0.000000,0.000000,0.417407,0.319829,0.322581,0.0,0.260870,0.300000,0.386792,0.078739,0.002474,0.409241,0.599842,0.132812


In [16]:
#x = df.drop(['domain1_score', 'essay','essay_id','essay_set'], axis=1)
# y = df['domain1_score']

x = df_normalized.drop(['domain1_score'],axis=1)
# # df['A']=df['A'].fillna(0.0).astype(int)
y = df['domain1_score'].fillna(0.0).astype(int)
x = np.array(x)
y = np.array(y)
where_are_NaNs = np.isnan(x)
x[where_are_NaNs] = 0
where_are_NaNs = np.isnan(y)
y[where_are_NaNs] = 0

In [17]:
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)
for train_index, test_index in kfold.split(x, y):
    X_train, X_test = x[train_index], x[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
logistic_reg = LogisticRegression()
logistic_reg.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [18]:
predictions = logistic_reg.predict(X_test)
print('Logistic regression classifier accuracy:', logistic_reg.score(X_test, y_test))

Logistic regression classifier accuracy: 0.379483224065


In [19]:
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)
for train_index, test_index in kfold.split(x, y):
    X_train, X_test = x[train_index], x[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
lin_reg = LinearRegression()
lin_reg.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [20]:
predictions = lin_reg.predict(X_test)
print('Linear regression classifier accuracy:', lin_reg.score(X_test, y_test))

Linear regression classifier accuracy: 0.270323426626


In [21]:
clf = svm.SVR(kernel="linear")
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
print('SVM (linear) Regressor Accuracy:', clf.score(X_test, y_test))

SVM (linear) Regressor Accuracy: 0.249024921898


In [22]:
model = RandomForestRegressor(random_state=0,n_estimators=100)
model.fit(X_train, y_train)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=1,
           oob_score=False, random_state=0, verbose=0, warm_start=False)

In [23]:
print ('Random Forest Regressor Accuracy:', model.score(X_test, y_test))

Random Forest Regressor Accuracy: 0.504137875794


In [24]:
from sklearn import svm
clf = svm.SVR(kernel="rbf")
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
print('SVM (rbf) Regressor Accuracy:', clf.score(X_test, y_test))

SVM (rbf) Regressor Accuracy: 0.315502457661


## LSTM

In [25]:
X = pd.read_excel('test2.xlsx')
X=X.drop(X.columns[0:2],axis=1)
y = pd.DataFrame(X['domain1_score'])

In [26]:
X.head()

,essay,domain1_score
0,"Dear local newspaper, I think effects computer...",6.666667
1,"Dear @CAPS1 @CAPS2, I believe that using compu...",7.500000
2,"Dear, @CAPS1 @CAPS2 @CAPS3 More and more peopl...",5.833333
3,"Dear Local Newspaper, @CAPS1 I have found that...",8.333333
4,"Dear @LOCATION1, I know having computers has a...",6.666667


In [27]:
y.head()

,domain1_score
0,6.666667
1,7.500000
2,5.833333
3,8.333333
4,6.666667


In [28]:
from keras.layers import Embedding, LSTM, Dense, Dropout, Lambda, Flatten
from keras.models import Sequential, load_model, model_from_config
import keras.backend as K

def get_model():
    model = Sequential([
        # 2D tensor for first layer: 1 timestep and 300 features
        LSTM(300, dropout=0.4, recurrent_dropout=0.4, input_shape=[1, 300], return_sequences=True),
        LSTM(64, recurrent_dropout=0.4),
        Dropout(0.5),
        Dense(1, activation='relu')
    ])

    model.compile(loss='mean_squared_error', optimizer='rmsprop', metrics=['mae'])
    model.summary()

    return model

In [30]:
import numpy as np
import nltk
import re
from nltk.corpus import stopwords
from gensim.models import Word2Vec

stopwords = set(stopwords.words('english'))

def essay_to_list(essay):
    # Remove the tags
    essay = re.sub("[^a-zA-Z]", " ", essay)
    words = essay.lower().split()
    return [w for w in words if not w in stopwords]

def essay_to_sentences(essay):
    tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
    raw_sentences = tokenizer.tokenize(essay.strip())
    sentences = []
    for raw_sentence in raw_sentences:
        if len(raw_sentence) > 0:
            sentences.append(essay_to_list(raw_sentence))
    return sentences

# Generate feature vector for the words
def get_feature_vector(words, model, num_features, vec_type="sum"):
    feature_vector = np.zeros((num_features,),dtype="float32")
    num_words = 0.
    index2word_set = set(model.wv.index2word)
    
    max_vec =  np.zeros((num_features,),dtype="float32")
    min_vec =  np.ones((num_features,),dtype="float32")

    for word in words:
        if word in index2word_set:
            num_words += 1
            max_vec = np.maximum(model[word], feature_vector)
            min_vec = np.minimum(model[word], feature_vector)
            feature_vector = np.add(feature_vector, model[word]) 
    
    # return min vector + max vector
    if vec_type == "min+max":
        return np.add(min_vec, max_vec) 
    
    # average of vectors
    elif vec_type == "average":
        return np.divide(feature_vector, num_words)

    # default: return sum of word2vec vectors
    return feature_vector

# Generate word vectors from the mdoel
def generate_essay_vectors(essays, model, num_features, vec_type="sum"):
    essayfeature_vectors = np.zeros((len(essays),num_features),dtype="float32")
    for (i, essay) in enumerate(essays):
        essayfeature_vectors[i] = get_feature_vector(essay, model, num_features, vec_type)
    return essayfeature_vectors

In [31]:
from sklearn.model_selection import KFold
from sklearn.linear_model import LinearRegression
from sklearn.metrics import cohen_kappa_score

def train_model(X, y, dataset, vec_type="sum"):
    count = 1
    results = []
    
    for train_set, test_set in dataset:
        print("Fold #", count)
        X_test, X_train, y_test, y_train = X.iloc[test_set], X.iloc[train_set], y.iloc[test_set], y.iloc[train_set]
        
        train_essays = X_train['essay']
        test_essays = X_test['essay']
        
        sentences = []
        
        for essay in train_essays:
            sentences += essay_to_sentences(essay)
                
        # Initialize variables for word2vec model
        num_features = 300 
        min_word_count = 40
        num_workers = 4
        context = 10
        downsampling = 1e-7

        # Train the word2vec model
        model = Word2Vec(sentences, workers=num_workers, size=num_features, min_count = min_word_count, window = context, sample = downsampling)
        model.init_sims(replace=True)
        
        # Generate training vectors
        clean_train_essays = []
        for essay_vec in train_essays:
            clean_train_essays.append(essay_to_list(essay_vec))
        train_vectors = generate_essay_vectors(clean_train_essays, model, num_features, vec_type)
        
        # Generate test vectors
        clean_test_essays = []
        for essay_vec in test_essays:
            clean_test_essays.append(essay_to_list( essay_vec))
        test_vectors = generate_essay_vectors(clean_test_essays, model, num_features, vec_type)
        
        train_vectors = np.array(train_vectors)
        test_vectors = np.array(test_vectors)

        # Reshape the train and test vectors to 3 dimensions - 1 represents one timestamp 
        train_vectors = np.reshape(train_vectors, (train_vectors.shape[0], 1, train_vectors.shape[1]))
        test_vectors = np.reshape(test_vectors, (test_vectors.shape[0], 1, test_vectors.shape[1]))
        
        # Call the LSTM to get the score predictions 
        lstm_model = get_model()
        lstm_model.fit(train_vectors, y_train, batch_size=64, epochs=50)
        y_pred = lstm_model.predict(test_vectors)
        
        # Round the prediction to the nearest integer
        y_pred = np.around(y_pred)
        
        # Evaluate the model: quadratic kappa score of predictions against human grading
        result = cohen_kappa_score(y_test.values, y_pred, weights='quadratic')
        print("QWK: ", result)
        results.append(result)
        
        count += 1

    return results

In [32]:
y['domain1_score']=df['domain1_score'].fillna(0.0).astype(int)
dataset = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)
dataset = dataset.split(X, y)
results_min_max = train_model(X, y, dataset, "min+max")
print("Average Quadratic Weighted Kappa after 5-fold cross validation for min + max word2vec ",np.around(np.array(results_min_max).mean(),decimals=4))

Fold # 1


/home/ayushi/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:36: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
/home/ayushi/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:37: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 1, 300)            721200    
_________________________________________________________________
lstm_2 (LSTM)                (None, 64)                93440     
_________________________________________________________________
dropout_1 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 65        
Total params: 814,705
Trainable params: 814,705
Non-trainable params: 0
_________________________________________________________________
Epoch 1/50
10377/10377 [==============================] - 9s 840us/step - loss: 5.6215 - mean_absolute_error: 1.7773
Epoch 2/50
10377/10377 [==============================] - 6s 594us/step - loss: 3.4326 - mean_absolute_error: 1.4382
Epoch 3/50
10377/10377 [==========

Epoch 17/50
10384/10384 [==============================] - 6s 535us/step - loss: 2.0171 - mean_absolute_error: 1.1064
Epoch 18/50
10384/10384 [==============================] - 6s 535us/step - loss: 1.9862 - mean_absolute_error: 1.0972
Epoch 19/50
10384/10384 [==============================] - 6s 535us/step - loss: 1.9875 - mean_absolute_error: 1.0924
Epoch 20/50
10384/10384 [==============================] - 6s 540us/step - loss: 1.9496 - mean_absolute_error: 1.0872
Epoch 21/50
10384/10384 [==============================] - 6s 536us/step - loss: 1.8827 - mean_absolute_error: 1.0646
Epoch 22/50
10384/10384 [==============================] - 6s 536us/step - loss: 1.8162 - mean_absolute_error: 1.0449
Epoch 23/50
10384/10384 [==============================] - 6s 535us/step - loss: 1.7871 - mean_absolute_error: 1.0352
Epoch 24/50
10384/10384 [==============================] - 6s 539us/step - loss: 1.7369 - mean_absolute_error: 1.0255
Epoch 25/50
10384/10384 [==============================]

10385/10385 [==============================] - 6s 581us/step - loss: 1.2540 - mean_absolute_error: 0.8713
Epoch 40/50
10385/10385 [==============================] - 6s 575us/step - loss: 1.2575 - mean_absolute_error: 0.8745
Epoch 41/50
10385/10385 [==============================] - 6s 587us/step - loss: 1.2479 - mean_absolute_error: 0.8626
Epoch 42/50
10385/10385 [==============================] - 6s 604us/step - loss: 1.2161 - mean_absolute_error: 0.8605
Epoch 43/50
10385/10385 [==============================] - 8s 728us/step - loss: 1.2400 - mean_absolute_error: 0.8640
Epoch 44/50
10385/10385 [==============================] - 7s 666us/step - loss: 1.1730 - mean_absolute_error: 0.8430
Epoch 45/50
10385/10385 [==============================] - 6s 555us/step - loss: 1.1432 - mean_absolute_error: 0.8349
Epoch 46/50
10385/10385 [==============================] - 5s 476us/step - loss: 1.1666 - mean_absolute_error: 0.8353
Epoch 47/50
10385/10385 [==============================] - 5s 479us/

/home/ayushi/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:38: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
